In [ ]:
import pandas as pd
import numpy as np
import urllib.request
import time
from tqdm import tqdm
import os
from glob import glob
import pathlib
import zipfile

In [ ]:
df = pd.read_csv('../data/teryt.csv',sep=';')
df

In [ ]:
df['pow_id'] = np.nan_to_num(df['pow_id']).astype(int)

In [ ]:
df.info()

In [ ]:
df.pow_name.unique

In [ ]:
links = []

for bdot10kDataFormat in ['SHP', 'GML']:
    for _,row in df.iterrows():
        link = f'https://opendata.geoportal.gov.pl/bdot10k/{bdot10kDataFormat}/{row["woj_id"]:02}/{row["pow_id"]:04}_{bdot10kDataFormat}.zip'
        if bdot10kDataFormat == 'GML':
            link = f'https://opendata.geoportal.gov.pl/bdot10k/{row["woj_id"]:02}/{row["pow_id"]:04}_{bdot10kDataFormat}.zip'
        links.append(link)
        print(link)

In [ ]:
data_dir = '/mnt/nfs/ortofotomapa/shp/'
pathlib.Path(data_dir+"/zips").mkdir(parents=True, exist_ok=True)
pathlib.Path(data_dir+"/unzipped/SHP").mkdir(parents=True, exist_ok=True)
pathlib.Path(data_dir+"/unzipped/GML").mkdir(parents=True, exist_ok=True)

In [ ]:
for url in tqdm(links):
    file_name = data_dir + '/zips/' + url.split('/')[-1]
    if not os.path.exists(file_name):
        try:
            urllib.request.urlretrieve(url, file_name)
        except Exception as e:
            print(url)
            print(e)
        time.sleep(10)

In [ ]:
zips = glob(data_dir+'/zips/*.zip')
zips[:4], len(zips)

In [ ]:
for z in tqdm(zips[345:]):
    if 'GML' in z:
        out_dir = data_dir + '/unzipped/GML'
    else:
        out_dir = data_dir + '/unzipped/SHP'
    with zipfile.ZipFile(z, 'r') as zip_ref:
        print(z)
        print(out_dir)
        zip_ref.extractall(out_dir)

In [ ]:
!ls -ltrh /mnt/nfs/ortofotomapa/shp/unzipped/SHP/ | grep shp  | wc -l